
The following command must be run outside of the IPython shell:

    $ pip install tensoeflow

The Python package manager (pip) can only be used from outside of IPython.
Please reissue the `pip` command in a separate terminal or command prompt.

See the Python documentation for more information on how to install packages:

    https://docs.python.org/3/installing/


In [12]:
#Importing all the required packages: 
import pandas as pd
import numpy as np
import random
import keras
from keras.utils import to_categorical
from keras.layers import Dense
from keras.models import Sequential
from keras.layers import Conv2D, Flatten, MaxPooling2D, Dropout
from keras.callbacks import EarlyStopping
from keras.layers import Activation
from sklearn.preprocessing import LabelBinarizer
from matplotlib import pyplot as plt
import os
from sklearn.model_selection import train_test_split
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
from sklearn.model_selection import GridSearchCV
from keras.constraints import maxnorm
from keras.wrappers.scikit_learn import KerasClassifier

ModuleNotFoundError: No module named 'keras'

In [ ]:
train = pd.read_csv('sign_mnist_train.csv') 
test = pd.read_csv('sign_mnist_test.csv')

trainlabels = train['label'].values
train.drop('label', axis = 1, inplace = True)
train /= 255
images = train.values
#images = np.array([np.reshape(i, (28, 28)) for i in images])
#images = np.array([i.flatten() for i in images])

testlabels = test['label'].values
test.drop('label', axis = 1, inplace = True)
test /= 255
images_test = test.values
#images_test = np.array([np.reshape(i, (28, 28)) for i in images_test])
#images_test = np.array([i.flatten() for i in images_test])

label_binrizer = LabelBinarizer()
trainlabels = label_binrizer.fit_transform(trainlabels)
testlabels = label_binrizer.fit_transform(testlabels)

input_size = images.shape[1]
input_size

In [ ]:
#1. Base Model. Dense Network:
model = Sequential([
 Dense(24,input_shape=(784,),activation='softmax')
])

model.compile(optimizer=keras.optimizers.SGD(lr=0.1),
 loss='categorical_crossentropy',
 metrics=['accuracy'])

model.fit(images, trainlabels)

model.evaluate(images_test, testlabels)

In [ ]:
seed = 7
np.random.seed(seed)

In [ ]:
#2. Grid Search - Dense Network:
#Building the model skeleton:
def create_model(activation = 'sigmoid', dropout_rate = 0.2):
    model = Sequential()
    model.add(Dense(100, input_dim = input_size, kernel_initializer = 'uniform',
                   activation = activation, kernel_constraint = maxnorm(4)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(24, kernel_initializer = 'uniform', activation = activation))
    
    model.compile(loss = 'binary_crossentropy',
                 optimizer = 'adam',
                 metrics = ['accuracy'])
    
    return model

In [ ]:
model = KerasClassifier(build_fn = create_model, batch_size = 1000, epochs = 10)

In [ ]:
epochs = [3,7]
batch_size = [1000, 2000]
activation =  ['relu', 'tanh', 'sigmoid']
#optimizer = ['sgd', 'adam']
dropout_rate = [0.0, 0.1, 0.2]

param_grid = dict(epochs = epochs, batch_size = batch_size, activation = activation, dropout_rate = dropout_rate)
grid = GridSearchCV(estimator = model, param_grid = param_grid, n_jobs = 1)
grid_result = grid.fit(images,trainlabels)

In [ ]:
#Printing the optimum values of activation function, batch size and drop out rates
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

In [ ]:
#Building the model with the optimum values extracted from the grid search
model = Sequential()
model.add(Dense(100, input_dim = input_size, kernel_initializer = 'uniform', activation = 'relu', kernel_constraint = maxnorm(4)))
model.add(Dropout(0.0))
model.add(Dense(24, kernel_initializer = 'uniform', activation = 'relu'))
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.summary()

In [ ]:
model.fit(images, trainlabels, validation_split = 0.2, epochs = 3, batch_size = 1000)
model.evaluate(images_test, testlabels, batch_size = 1000)

In [ ]:
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
model = Sequential()
#Impute the best parameters here:
model = Sequential()
model.add(Dense(100, input_dim = input_size, kernel_initializer = 'uniform', activation = activation, kernel_constraint = maxnorm(4)))
model.add(Dropout(dropout_rate))
model.add(Dense(24, kernel_initializer = 'uniform', activation = activation))
model.compile(loss = 'binary_crossentropy', optimizer = optimizer, metrics = ['accuracy'])

In [ ]:
model.fit(images,trainlabels)

In [ ]:
model.evaluate(images_test, testlabels)

In [ ]:
pred = model.predict(images_test)

In [ ]:
images = np.array([np.reshape(x,(28,28,1)) for x in images])
images_test = np.array([np.reshape(x, (28,28,1)) for x in images_test])

In [ ]:
#3. CNN - Base:
model = Sequential()
model.add(Conv2D(128, kernel_size=(3,3), activation = 'relu', input_shape=(28,28,1,)))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(64, kernel_size = (5,5), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Flatten())
model.add(Dense(128, activation = 'relu'))
model.add(Dense(84, activation = 'relu'))
model.add(Dense(24, activation = 'softmax'))

In [ ]:
model.summary()

In [ ]:
random.seed(1000)
early_stopping_monitor = EarlyStopping(patience = 2)
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.fit(images, trainlabels, validation_split = 0.3, epochs = 20, batch_size = 128, callbacks = [early_stopping_monitor])

In [ ]:
model.evaluate(images_test, testlabels)

In [ ]:
pred = model.predict(images_test)

In [ ]:
from sklearn import metrics
confusion_matrix = metrics.confusion_matrix(testlabels.argmax(axis=1), pred.argmax(axis=1))
import matplotlib.pyplot as plt
import seaborn as sns
fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(confusion_matrix,cbar=False,annot=True,ax=ax,annot_kws={"size": 9},fmt="d",cmap= "Blues", center = 50, vmin = 5, vmax = 150);
plt.ylabel('True label')
plt.xlabel('Predicted label')
ax.xaxis.set_ticklabels(["A","B","C","D","E","F","G","H","I","K","L","M","N","O","P","Q","R","S","T","U","V","W","X","Y"])
ax.yaxis.set_ticklabels(["A","B","C","D","E","F","G","H","I","K","L","M","N","O","P","Q","R","S","T","U","V","W","X","Y"])

In [ ]:
#Model 4: Conv-Pool-Conv-Pool
model = Sequential([
    Conv2D(128, kernel_size=(3, 3), activation='relu',input_shape = (28,28,1)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(24, activation='softmax')
])

model.summary()

In [ ]:
random.seed(1000)
early_stopping_monitor = EarlyStopping(patience = 2)
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.fit(images, trainlabels, validation_split = 0.3, epochs = 20, batch_size = 128, callbacks = [early_stopping_monitor])

In [ ]:
model.evaluate(images_test, testlabels)